In [2]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import seaborn as sns
import scienceplots
plt.style.use('science')
from pylab import xticks,yticks,np
import numpy as np
import matplotlib.pyplot as plt
from sklearn.mixture import GaussianMixture
from scipy.stats import gaussian_kde
from mpl_toolkits.axes_grid1 import make_axes_locatable
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Image
from scipy.stats import gaussian_kde, norm
from sklearn.mixture import GaussianMixture

from utils_f4f import si_format
import json

In [ ]:
# Import data 
assay = 'nor_package'
array = 'AA_sequence'

df_all = pd.read_csv('../data/1st_780w_packseq_aa.tsv',delimiter='\t')
df_all = df_all.rename(columns={'aa':'AA_sequence'})
df = df_all[['AA_sequence', 'nor_package']]
df= df.sort_values(by='nor_package')

#Fill in the nan data and replace the inf value
df['nor_package'] = df['nor_package'].fillna(0)  
df['nor_package'] = df['nor_package'].replace([np.inf, -np.inf], np.nan)
df = df.dropna(subset=['nor_package','AA_sequence'])

#Stratified sampling
df = df.iloc[::5]
df.shape

In [ ]:
nor_package = df_all[np.isfinite(df_all[assay])][assay]
data = np.array(nor_package) 
data = data.reshape(-1, 1)  # Convert the data into a column vector

# Fit the data using GaussianMixture
n_components = 2 # Set the number of components for the hybrid model
gmm = GaussianMixture(n_components=n_components)
gmm.fit(data)

# Get the mean and covariance matrix for each component
means = gmm.means_
covariances = gmm.covariances_

# Generate the fitted distribution
x = np.linspace(-10, 5, 10000)
x = x.reshape(-1, 1)
y = np.exp(gmm.score_samples(x))

fig,ax = plt.subplots(figsize=(2,1.7),dpi=100)
# Plot the raw data and the distribution after fitting
plt.hist(data, bins=100, density=True, alpha=0.6, color = '#BAB3A3',label='Prediction')

# Plot the normal distribution for each component
for i in range(n_components):
    component = np.exp(-(x - means[i]) ** 2 / (2 * covariances[i]))
    component /= np.sqrt(2 * np.pi * covariances[i])
    component *= gmm.weights_[i]
    if i == 0:
        ax.plot(x, component, '-', label=f'Distribution {i + 1}', color='#F66E68', alpha=1)
        ax.fill_between(x.flatten(), 0, component.flatten(), color='#F66E68', alpha=0.4)
    else:
        ax.plot(x, component, '-', label=f'Distribution {i + 1}', color='#457B9D', alpha=1)
        ax.fill_between(x.flatten(), 0, component.flatten(), color='#457B9D', alpha=0.4)


plt.title('Truth_distribution',fontsize=10)
ax.tick_params(axis='both', which='both', length=2,labelsize=8)
ax.set_xticks([-5, 0, 5]); 
# ax.set_yticks([])
plt.ylabel('Density',labelpad=2,fontsize=10)
plt.xlabel('Nor_package',labelpad=2,fontsize=10)
plt.legend(fontsize=7,frameon=False,loc='best')
plt.show()

In [19]:


def onehot_flatten(integer_vec, alphabet):
    num_labels = len(alphabet)
    one_hot = np.zeros((len(integer_vec), num_labels))
    one_hot[np.arange(len(integer_vec)), integer_vec] = 1
    return one_hot.flatten()

def seq_to_onehot(char_sequences, alphabet):
    char_to_int = {c: i for i, c in enumerate(alphabet)}
    integer_encoded = [[char_to_int[char] for char in seq] for seq in char_sequences]
    integer_encoded = pd.DataFrame(integer_encoded)
    
    one_hot_encoded_list = []
    for index, row in integer_encoded.iterrows():
        one_hot_encoded_list.append(onehot_flatten(row, alphabet))
    
    one_hot_encoded = pd.DataFrame(one_hot_encoded_list, dtype='int')
    
    return one_hot_encoded, integer_encoded

# Define alphabet
alphabet = 'ACDEFGHIKLMNPQRSTVWY'
# Apply one-hot encoding
X, integer_encoded = seq_to_onehot(list(df['AA_sequence']), alphabet=alphabet)

In [ ]:
X

In [5]:
aa_values_list  = X
nor_package_values_list = df['nor_package']



In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

# Example training data
X_train = np.array(aa_values_list)
y_train = np.array(nor_package_values_list)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Standardize the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Build the neural network model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.BatchNormalization(),  # Batch Normalization layer
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.BatchNormalization(),  # Batch Normalization layer
    tf.keras.layers.Dense(1, activation='relu')
])

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005), loss='mean_squared_error')

# Implement early stopping
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model
model.fit(X_train_scaled, y_train, epochs=20, batch_size=16, validation_split=0.2, callbacks=[early_stopping], verbose=1)

# Make predictions on the test set
predicted_activity = model.predict(X_test_scaled).flatten()

# Evaluate the model
mse = mean_squared_error(y_test, predicted_activity)
r2 = r2_score(y_test, predicted_activity)

print("Mean Squared Error:", mse)
print("R-squared:", r2)


In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score


# Reshape X_train to have a time dimension

X_train = np.array(aa_values_list)
y_train = np.array(nor_package_values_list)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Standardize the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.reshape(-1, 1)).reshape(X_train.shape)
X_test_scaled = scaler.transform(X_test.reshape(-1, 1)).reshape(X_test.shape)

# Build the improved RNN model
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(128, activation='relu', input_shape=(X_train.shape[1], 1), return_sequences=True),
    tf.keras.layers.LSTM(64, activation='relu', return_sequences=True),
    tf.keras.layers.LSTM(32, activation='relu'),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1, activation='linear')
])

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='mean_squared_error')

# Early stopping callback
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model
model.fit(X_train_scaled, y_train, epochs=10, batch_size=32, validation_split=0.2, callbacks=[early_stopping], verbose=1)

# Make predictions on the test set
predicted_activity = model.predict(X_test_scaled).flatten()

# Evaluate the model
mse = mean_squared_error(y_test, predicted_activity)
r2 = r2_score(y_test, predicted_activity)

print("Mean Squared Error:", mse)
print("R-squared:", r2)


In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score


# Reshape X_train to have a time dimension

X_train = np.array(aa_values_list)
y_train = np.array(nor_package_values_list)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Standardize the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.reshape(-1, 1)).reshape(X_train.shape)
X_test_scaled = scaler.transform(X_test.reshape(-1, 1)).reshape(X_test.shape)

# Build the improved RNN model
model = tf.keras.Sequential([
    tf.keras.layers.GRU(128, activation='relu', input_shape=(X_train.shape[1], 1), return_sequences=True),
    tf.keras.layers.GRU(64, activation='relu', return_sequences=True),
    tf.keras.layers.GRU(32, activation='relu'),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1, activation='linear')
])

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='mean_squared_error')

# Early stopping callback
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model
model.fit(X_train_scaled, y_train, epochs=50, batch_size=32, validation_split=0.2, callbacks=[early_stopping], verbose=1)

# Make predictions on the test set
predicted_activity = model.predict(X_test_scaled).flatten()

# Evaluate the model
mse = mean_squared_error(y_test, predicted_activity)
r2 = r2_score(y_test, predicted_activity)

print("Mean Squared Error:", mse)
print("R-squared:", r2)





In [ ]:


df1 = pd.DataFrame(predicted_activity)
df2 = pd.DataFrame(y_test)

random_indices = np.random.choice(len(df1), size=10000, replace=False)
df1_sampled = df1.iloc[random_indices]
df2_sampled = df2.iloc[random_indices]

# 对数据进行归一化
scaler = StandardScaler()
x_scaled = scaler.fit_transform(df1_sampled.values.reshape(-1, 1)).flatten()
y_scaled = scaler.fit_transform(df2_sampled.values.reshape(-1, 1)).flatten()
xy = np.vstack([x_scaled, y_scaled])
z = gaussian_kde(xy)(xy)
idx = z.argsort()
x_scaled, y_scaled, z = x_scaled[idx], y_scaled[idx], z[idx]
plt.style.use('nature')
fig, ax = plt.subplots(figsize=(3, 3), dpi=300)
scatter = ax.scatter(x_scaled, y_scaled, marker='o', c=z, edgecolors='none', s=10, cmap='Spectral_r', alpha=0.7)

ax.set_xlabel('Turth (Normalized)')
ax.set_ylabel('Predictions (Normalized)')
ax.set_title('Correlation +  GRU')

correlation = df1.corrwith(df2)
print("结果相关性：", correlation)
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.1)
cbar = fig.colorbar(scatter, cax=cax, label='Frequency')
ax.text(0.3, 0.9, f'Correlation: {correlation.values[0]:.4f}', transform=ax.transAxes, ha='center', va='center', bbox=dict(facecolor='white', alpha=0.8))
ax.plot([-2, 2], [-2, 2], ls="--", )

plt.show()


plt.style.use('nature')
data = np.array(df1) 
data = data.reshape(-1, 1)  # 将数据转换成列向量的形式

# 使用GaussianMixture拟合数据
n_components = 2 # 设置混合模型的分量个数
gmm = GaussianMixture(n_components=n_components)
gmm.fit(data)

# 获取每个分量的均值和协方差矩阵
means = gmm.means_
covariances = gmm.covariances_

# 生成拟合后的分布
x = np.linspace(-7, 5, 100000)
x = x.reshape(-1, 1)
y = np.exp(gmm.score_samples(x))

fig,ax = plt.subplots(figsize=(3,2.4),dpi=300)
# 绘制原始数据和拟合后的分布
plt.hist(data, bins=100, density=True, alpha=0.6, color = '#BAB3A3',label='Prediction')
#plt.plot(x, y, '-r', label='GMM fit')


# 绘制每个分量的正态分布
for i in range(n_components):
    component = np.exp(-(x - means[i]) ** 2 / (2 * covariances[i]))
    component /= np.sqrt(2 * np.pi * covariances[i])
    component *= gmm.weights_[i]
    if i == 0:
        ax.plot(x, component, '-', label=f'Distribution {i + 1}', color='#F66E68', alpha=1)
        ax.fill_between(x.flatten(), 0, component.flatten(), color='#F66E68', alpha=0.4)
    else:
        ax.plot(x, component, '-', label=f'Distribution {i + 1}', color='#457B9D', alpha=1)
        ax.fill_between(x.flatten(), 0, component.flatten(), color='#457B9D', alpha=0.4)

# ax.spines['top'].set_visible(False)
# ax.spines['right'].set_visible(False)
# ax.spines['left'].set_visible(False)
plt.title('Prediction Distribution(KDE)   GRU',fontsize=7)
# ax.tick_params(axis='both', which='both', length=0)
# ax.set_yticks([])
plt.ylabel('Density')
plt.xlabel('Nor_package')
plt.legend()
plt.show()

In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# Reshape X_train to have a time dimension

X_train = np.array(aa_values_list)
y_train = np.array(nor_package_values_list)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Standardize the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.reshape(-1, 1)).reshape(X_train.shape)
X_test_scaled = scaler.transform(X_test.reshape(-1, 1)).reshape(X_test.shape)

# Build the improved RNN model
model = tf.keras.Sequential([
    tf.keras.layers.SimpleRNN(128, activation='relu', input_shape=(X_train.shape[1], 1), return_sequences=True),
    tf.keras.layers.SimpleRNN(64, activation='relu', return_sequences=True),
    tf.keras.layers.SimpleRNN(32, activation='relu') ,
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1, activation='linear')
])

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='mean_squared_error')

# Early stopping callback
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model
model.fit(X_train_scaled, y_train, epochs=100, batch_size=32, validation_split=0.2, callbacks=[early_stopping], verbose=1)

model


In [ ]:
# Make predictions on the test set
predicted_activity = model.predict(X_test_scaled).flatten()

# Evaluate the model
mse = mean_squared_error(y_test, predicted_activity)
r2 = r2_score(y_test, predicted_activity)

print("Mean Squared Error:", mse)
print("R-squared:", r2)